In [1]:
# Importamos librerías Pandas y NumPy

import pandas as pd
import numpy as np

In [2]:
# Leemos el dataset

complete = pd.read_csv("../data/f18/complete.csv", encoding = 'utf8')
complete.head()

,ID,name,full_name,club,club_logo,special,age,league,birth_date,height_cm,...,prefers_cb,prefers_lb,prefers_lwb,prefers_ls,prefers_lf,prefers_lam,prefers_lcm,prefers_ldm,prefers_lcb,prefers_gk
0,20801,Cristiano Ronaldo,C. Ronaldo dos Santos Aveiro,Real Madrid CF,https://cdn.sofifa.org/18/teams/243.png,2228,32,Spanish Primera División,1985-02-05,185.0,...,False,False,False,False,False,False,False,False,False,False
1,158023,L. Messi,Lionel Messi,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2158,30,Spanish Primera División,1987-06-24,170.0,...,False,False,False,False,False,False,False,False,False,False
2,190871,Neymar,Neymar da Silva Santos Jr.,Paris Saint-Germain,https://cdn.sofifa.org/18/teams/73.png,2100,25,French Ligue 1,1992-02-05,175.0,...,False,False,False,False,False,False,False,False,False,False
3,176580,L. Suárez,Luis Suárez,FC Barcelona,https://cdn.sofifa.org/18/teams/241.png,2291,30,Spanish Primera División,1987-01-24,182.0,...,False,False,False,False,False,False,False,False,False,False
4,167495,M. Neuer,Manuel Neuer,FC Bayern Munich,https://cdn.sofifa.org/18/teams/21.png,1493,31,German Bundesliga,1986-03-27,193.0,...,False,False,False,False,False,False,False,False,False,True


In [3]:
# Obtenemos las columnas que nos interesan

columns = ["ID", "pac", "sho", "pas", "dri", "def", "phy"]
df = complete[columns].copy()
df.head()

,ID,pac,sho,pas,dri,def,phy
0,20801,90,93,82,90,33,80
1,158023,89,90,86,96,26,61
2,190871,92,84,79,95,30,60
3,176580,82,90,79,87,42,81
4,167495,91,90,95,89,60,91


In [4]:
# Escribimos el resultado en CSV

df.to_csv('../data/output/features_2018.csv', sep=',', columns = columns, encoding='utf-8')